<a href="https://colab.research.google.com/github/Nastia12345678/PAS/blob/main/trace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **trace**: трассировка выполнения программы

Модуль **trace** позволяет наблюдать за выполнением инструкций, создавать отчеты
о покрытии кода и исследовать отношения между функциями, вызывающими
одна другую.

# Пример программы 

In [ ]:
#trace_example / main.py 
def main():
    print('This is the main program.')
    recurse(2)

if __name__ == '__main__':
    main()

In [64]:
#trace_example / recurse.py 
def recurse(level):
    print('recurse({})'.format(level))
    if level:
        recurse(level - 1)


def not_called():
    print('This function is never called.')


# Отслеживание выполнения 


In [ ]:
$ python3 -m trace --ignore-dir=.../lib/python3.7 \
--trace trace_example/main.py

 --- modulename: main, funcname: <module>
main.py(7): """
main.py(10): from recurse import recurse
 --- modulename: recurse, funcname: <module>
recurse.py(7): """
recurse.py(11): def recurse(level):
recurse.py(17): def not_called():
main.py(13): def main():
main.py(18): if __name__ == '__main__':
main.py(19):     main()
 --- modulename: main, funcname: main
main.py(14):     print('This is the main program.')
This is the main program.
main.py(15):     recurse(2)
 --- modulename: recurse, funcname: recurse
recurse.py(12):     print('recurse({})'.format(level))
recurse(2)
recurse.py(13):     if level:
recurse.py(14):         recurse(level - 1)
 --- modulename: recurse, funcname: recurse
recurse.py(12):     print('recurse({})'.format(level))
recurse(1)
recurse.py(13):     if level:
recurse.py(14):         recurse(level - 1)
 --- modulename: recurse, funcname: recurse
recurse.py(12):     print('recurse({})'.format(level))
recurse(0)
recurse.py(13):     if level:

# **Вызов отношений**

**trace** собирает и сообщает о взаимосвязях между функциями, которые вызывают друг друга.

Для простого списка вызываемых функций используйте --listfuncs.

In [ ]:
$ python3 -m trace --listfuncs trace_example/main.py | \
grep -v importlib

This is the main program.
recurse(2)
recurse(1)
recurse(0)

functions called:
filename: trace_example/main.py, modulename: main, funcname:
<module>
filename: trace_example/main.py, modulename: main, funcname:
main
filename: trace_example/recurse.py, modulename: recurse,
funcname: <module>
filename: trace_example/recurse.py, modulename: recurse,
funcname: recurse

Для получения дополнительных сведений о том, кто звонит, используйте **--trackcalls**.

In [ ]:
$ python3 -m trace --listfuncs --trackcalls \
trace_example/main.py | grep -v importlib

This is the main program.
recurse(2)
recurse(1)
recurse(0)

calling relationships:

*** .../lib/python3.7/trace.py ***
  --> trace_example/main.py
    trace.Trace.runctx -> main.<module>

  --> trace_example/recurse.py


*** trace_example/main.py ***
    main.<module> -> main.main
  --> trace_example/recurse.py
    main.main -> recurse.recurse

*** trace_example/recurse.py ***
    recurse.recurse -> recurse.recurse

# **Интерфейс программирования**

In [69]:
#trace_run.py 
import trace

tracer = trace.Trace(count=False, trace=True)
tracer.run('recurse(2)')

 --- modulename: iostream, funcname: write
iostream.py(383):         if self.echo is not None:
iostream.py(391):         if self.pub_thread is None:
iostream.py(395):             if not isinstance(string, unicode_type):
iostream.py(398):             is_child = (not self._is_master_process())
 --- modulename: iostream, funcname: _is_master_process
iostream.py(308):         return os.getpid() == self._master_pid
iostream.py(400):             self.pub_thread.schedule(lambda : self._buffer.write(string))
 --- modulename: iostream, funcname: schedule
iostream.py(200):         if self.thread.is_alive():
 --- modulename: threading, funcname: is_alive
threading.py(1112):         assert self._initialized, "Thread.__init__() not called"
threading.py(1113):         if self._is_stopped or not self._started.is_set():
 --- modulename: threading, funcname: is_set
threading.py(508):         return self._flag
threading.py(1115):         self._wait_for_tstate_lock(False)
 --- modulename: threading, func

In [70]:
#trace_runfunc.py 
import trace

tracer = trace.Trace(count=False, trace=True)
tracer.runfunc(recurse, 2)

 --- modulename: iostream, funcname: write
iostream.py(383):         if self.echo is not None:
iostream.py(391):         if self.pub_thread is None:
iostream.py(395):             if not isinstance(string, unicode_type):
iostream.py(398):             is_child = (not self._is_master_process())
 --- modulename: iostream, funcname: _is_master_process
iostream.py(308):         return os.getpid() == self._master_pid
iostream.py(400):             self.pub_thread.schedule(lambda : self._buffer.write(string))
 --- modulename: iostream, funcname: schedule
iostream.py(200):         if self.thread.is_alive():
 --- modulename: threading, funcname: is_alive
threading.py(1112):         assert self._initialized, "Thread.__init__() not called"
threading.py(1113):         if self._is_stopped or not self._started.is_set():
 --- modulename: threading, funcname: is_set
threading.py(508):         return self._flag
threading.py(1115):         self._wait_for_tstate_lock(False)
 --- modulename: threading, func

# **Сохранение данных результатов**

Данные
можно сохранить явным образом, используя экземпляр CoverageResults из объекта Trace.


In [71]:
#trace_CoverageResults.py 
import trace

tracer = trace.Trace(count=True, trace=False)
tracer.runfunc(recurse, 2)

results = tracer.results()
results.write_results(coverdir='coverdir2')

recurse(2)
recurse(1)
recurse(0)


Этот пример сохраняет результаты покрытия в каталог coverdir2.

Чтобы сохранить данные о количестве вызовов функций для генерации отчетов, следует использовать аргументы infile и outfile при вызове конструктора
Trace

In [72]:
#trace_report.py 
import trace

tracer = trace.Trace(count=True,
                     trace=False,
                     outfile='trace_report.dat')
tracer.runfunc(recurse, 2)

report_tracer = trace.Trace(count=False,
                            trace=False,
                            infile='trace_report.dat')
results = tracer.results()
results.write_results(summary=True, coverdir='/tmp')

recurse(2)
recurse(1)
recurse(0)
lines   cov%   module   (path)
  252     7%   ipykernel.iostream   (/usr/local/lib/python3.6/dist-packages/ipykernel/iostream.py)
  596     1%   threading   (/usr/lib/python3.6/threading.py)
  252     1%   zmq.sugar.socket   (/usr/local/lib/python3.6/dist-packages/zmq/sugar/socket.py)


Имя входного файла, из которого необходимо прочитать ранее сохраненные
данные, задается c помощью аргумента infile, а имя выходного файла, в который необходимо записать новые результаты после выполнения трассировки, —
c помощью аргумента outfile. В случае совпадения имен входного и выходного
файлов предыдущий код обновляет файл кумулятивными данными.

# **Опции**


*   count — булево значение. Включает подсчет количества вызовов каждой
инструкции. По умолчанию принимает значение True.

*  countfuncs — булево значение. Включает вывод списка функций, вызываемых в процессе выполнения. По умолчанию принимает значение False.

*   countcallers — булево значение. Включает отслеживание вызывающих
и вызываемых объектов. По умолчанию принимает значение True.

*   ignoremods — последовательность. Список модулей или пакетов, которые
будут игнорироваться при отслеживании покрытия кода. Значением по
умолчанию является пустой кортеж.

*   ignoredirs — последовательность. Список каталогов, содержащих модули
или пакеты, которые следует игнорировать. По умолчанию — пустой кортеж.

*   infile — имя файла, содержащего кешированные значения счетчиков. По
умолчанию принимает значение None.

*   infile — имя файла, содержащего кешированные значения счетчиков. По
умолчанию принимает значение None.







